# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from A3C_Agent import *
from ENV_DETAILS import *

2023-10-23 10:56:40.192715: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using TensorFlow backend
Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\Pendulum\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/Pendulum/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_a3c/' # Linux

ENV = "Pendulum-v1"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

NameError: name 'os' is not defined

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<PassiveEnvChecker<PendulumEnv<Pendulum-v1>>>>>

In [4]:
s = env.observation_space.sample()
s

array([ 0.81201607,  0.7711965 , -1.8408298 ], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1. -1. -8.], [1. 1. 8.], (3,), float32), (3,))

In [6]:
s = env.reset()
s[0]

array([-0.1373922, -0.9905167,  0.9398161], dtype=float32)

In [7]:
s = env.reset()[0]
env.action_space, env.action_space, env.step(s)

(Box(-2.0, 2.0, (1,), float32),
 Box(-2.0, 2.0, (1,), float32),
 (array([-0.7096961 , -0.7045079 , -0.95374924], dtype=float32),
  -5.355570393613848,
  False,
  False,
  {}))

# Advantage Asynch Actor-Critic 

This time we will implement the A3C not considering Actor and Critic as part of the same network

### Training/Hyperparam run

In [8]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_A3C/"

if TUNING_TYPE == "MANUAL":
    params = {}
    params["n_enviroment"] =[20]
    params["discount"] =[0.96, 0.97]
    params["end_of_episode"] = [400]
    params["learning_rate"] = [0.001]
    params["entropy_factor"] = [ 0.01, 0.05]
    params["epsilon"] = [1]
    params["boltzman_factor"] = [1]
    params["exploration_technique"] = ['soft', 'epsilon']
    params["training_steps"] = [3000000]
    params["dense_units"] = [32, 128]
    params["time_to_update"] = [400]
    params["use_LSTM"] =[False]


    hyperparam_combination = list(itertools.product(*list(params.values())))
    

    try:

        files = [name for name in os.listdir(logs_dir) if os.path.isfile(os.path.join(logs_dir, name)) and name != "logfile.txt" and name != "merged_results.json"]
        if len(files) >= 1: merge_JsonFiles(main_hyper_dir, logs_dir, files)

        res_file = logs_dir+"merged_results.json"
        def without_keys(d, keys):
            return {x: d[x] for x in d if x not in keys}


        if os.path.isfile(res_file):
            with open(res_file, 'r') as f:
                complete_file = json.load(f)

            newlist = sorted(complete_file, key=lambda d: d['mean_rewards'], reverse=True) 
            params = []
            for i, f in enumerate(newlist):
                label = "disc : "+str(f['discount'])+" | "+"lr : "+str(f['learning_rate'])+" | "+"entropy : "+str(f['entropy_factor'])+" | "+"update : "+str(f['time_to_update'])
                plt.figure(figsize=[35,4])
                plt.plot(f['rewards'], label = label)
                plt.legend()
                max_mean_reward = f['mean_rewards']
                params.append(f)
                print(without_keys(f,"rewards"))
                if i == 10:
                    break
            plt.title("Evaluation rewards"); plt.grid()
            plt.show()

    except:
        pass



In [10]:
num_layers = 3
dense = [32 for _ in range(num_layers)]
actorcritic_agent = ContinuousActorCriticAgent(
                    env = env, 
                    d = dense,
                    lstm_unit = 32,
                    use_LSTM = False,
                    sigma_noise = 0.0001,
                    num_layers = num_layers
                    )
obs = env.reset()[0]
obs = obs.reshape((1,env.observation_space.shape[0]))
actorcritic_agent(obs, inference = True)
actorcritic_agent.summary()

Model: "continuous_actor_critic_agent"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 v (Dense)                   multiple                  33        
                                                                 
 lstm (LSTM)                 multiple                  0 (unused)
                                                                 
 actor_mu (Dense)            multiple                  33        
                                                                 
 actor_sigma (Dense)         multiple                  33        
                                                                 
 inp_0 (Dense)               multiple                  128       
                                                                 
 inp_1 (Dense)               multiple                  1056      
                                                                 
 inp_2 (Dense)               multiple

In [11]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_A3C"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel(hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_soft/", 
                          end_of_episode = EPISODES, n_enviroment = 10, 
                  evaluation_epoch = 2000, training_steps = 500000,
                  sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                  discount_min = 0.90, discount_max = 0.99,
                  entropy_min = 0.00001, entropy_max = 0.01,
                  lr_min = 0.000001, lr_max = 0.005,
                  dense_min = 32, dense_max = 300,
                  lstm_min = 32, lstm_max = 128,
                  time_to_update_min = 100, time_to_update_max=600,
                  environment_name=ENV,
                  continuous_actions_space= True,
                  reward_scaler = 16.2736044,
                  n_dense_layers = 4
                  ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
   
else : 
    
        print("Acquiring parameters ....")
        writer= "Training/fit_A3C/"


        exploration_tech = "soft"
        discount = 0.99
        learning_rate = 0.00001
        entropy_factor = 0.001
        dense_units = [256]
        sigma_noise =  1e-5

        n_enviroment = 30
        end_ep = 1000
        ep = 1
        bolt_fact = 1
        training_steps = 4000000

        time_to_update= 100
        lstm_units= 32
        end_of_episode = EPISODES
        save_factor=50000
        sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
        sucess_criteria_value = SUCESS_CRITERIA_VALUE
        environment_name = ENV
        reward_scaler = 16.2736044
        continuous_space_actions = True
        return_agent = True
        use_LSTM = False

        env_model = run_training(training_steps, learning_rate, entropy_factor, exploration_tech, discount, time_to_update, dense_units, lstm_units, 
                        n_enviroment, writer, use_LSTM, end_of_episode, save_factor, sucess_criteria_epochs , sucess_criteria_value , 
                        environment_name, reward_scaler, continuous_space_actions, sigma_noise, return_agent)
            

Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft/tuner0.json


In [12]:

exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    n_env = 30
    end_ep = 1000
    ep = 1
    bolt_fact = 1
    train_steps = 5000000
    disc = trials.hyperparameters.values["discount"]
    lr = trials.hyperparameters.values["learning_rate"]
    entropy_fact = trials.hyperparameters.values["entropy_factor"]
    n_hidden_layer_hp = trials.hyperparameters.values["n_dense_layers"]

    d = []
    for i in range(n_hidden_layer_hp):
        d.append(trials.hyperparameters.values['dense_units_'+str(i)])

    sigma_noise = 1e-5
    ulstm = False

Trial id :28 | Score :-10.882266520567075 --> {'discount': 0.91, 'entropy_factor': 0.004869523196580806, 'learning_rate': 6.325876829563758e-05, 'n_dense_layers': 1, 'dense_units_0': 164, 'dense_units_1': 177, 'dense_units_2': 86, 'dense_units_3': 134}
Trial id :11 | Score :-45.21249650747056 --> {'discount': 0.92, 'entropy_factor': 0.0025030534967846957, 'learning_rate': 0.00136942884893527, 'n_dense_layers': 2, 'dense_units_0': 227, 'dense_units_1': 129, 'dense_units_2': 101}
Trial id :09 | Score :-68.6664493532472 --> {'discount': 0.93, 'entropy_factor': 0.00046711278429575583, 'learning_rate': 0.0001548827581762531, 'n_dense_layers': 2, 'dense_units_0': 151, 'dense_units_1': 69, 'dense_units_2': 147}


In [13]:
val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_A3C"

exploration_tech ="soft"
project_name = "keras_tunning_"+exploration_tech

tuner = kt.BayesianOptimization(
        MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/A3C_"+exploration_tech+"/", exploration_tech =exploration_tech, 
                        end_of_episode = EPISODES, n_enviroment = 10, 
                  evaluation_epoch = 2000, training_steps = 500000,
                  sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                  discount_min = 0.90, discount_max = 0.99,
                  entropy_min = 0.00001, entropy_max = 0.01,
                  lr_min = 0.000001, lr_max = 0.005,
                  dense_min = 32, dense_max = 300,
                  lstm_min = 32, lstm_max = 128,
                  time_to_update_min = 100, time_to_update_max=600,
                  environment_name=ENV,
                  continuous_actions_space= True,
                  reward_scaler = 16.2736044
                  ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
    )

for i, trials in enumerate(tuner.oracle.get_best_trials(num_trials=3)):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    
    if i ==0:
        print("reading...")
        env_model = tuner.get_best_models()[0]
        discount = trials.hyperparameters.values["discount"]
        learning_rate = trials.hyperparameters.values["learning_rate"]
        entropy_factor = trials.hyperparameters.values["entropy_factor"]
        n_hidden_layer_hp = trials.hyperparameters.values["n_dense_layers"]

        dense_units = []
        for i in range(n_hidden_layer_hp):
            dense_units.append(trials.hyperparameters.values['dense_units_'+str(i)])
        sigma_noise = 1e-5


final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech, sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, reward_scaler = 16.2736044 ,   video_name = "./A3C_"+exploration_tech+"_video.mp4", continuous_action_space= True)
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

Reloading Tuner from Hyperparam_kt_A3C/keras_tunning_soft/tuner0.json
Trial id :28 | Score :-10.882266520567075 --> {'discount': 0.91, 'entropy_factor': 0.004869523196580806, 'learning_rate': 6.325876829563758e-05, 'n_dense_layers': 1, 'dense_units_0': 164, 'dense_units_1': 177, 'dense_units_2': 86, 'dense_units_3': 134}
reading...
Trial number :  30
Trial id :11 | Score :-45.21249650747056 --> {'discount': 0.92, 'entropy_factor': 0.0025030534967846957, 'learning_rate': 0.00136942884893527, 'n_dense_layers': 2, 'dense_units_0': 227, 'dense_units_1': 129, 'dense_units_2': 101}
Trial id :09 | Score :-68.6664493532472 --> {'discount': 0.93, 'entropy_factor': 0.00046711278429575583, 'learning_rate': 0.0001548827581762531, 'n_dense_layers': 2, 'dense_units_0': 151, 'dense_units_1': 69, 'dense_units_2': 147}
Moviepy - Building video ./A3C_soft_video.mp4.
Moviepy - Writing video ./A3C_soft_video.mp4



Moviepy - Done !
Moviepy - video ready ./A3C_soft_video.mp4
Final mean reward ' soft ': -11.025725769174128
